<a href="https://colab.research.google.com/github/Nandini-55/AI-Machine_Learning_Journey/blob/main/Unsupervised%20Machine%20Learning/CNN_irisDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import & setup
import numpy as np   # Numerical arrays +RNG seeding
import tensorflow as tf  # alternative pytorch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split   #Train/validation/test Splitting
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt # alternative seaborn

In [ ]:
#fix random seeds so results are (reasonably) reproducible across runs
np.random.seed(42)  #Seed Numpy's random no. generator
tf.random.set_seed(42) #Seed tensorflow's random no. generator

In [ ]:
#  2) Load & prepare the data
iris = load_iris()   # Loads features (iris.data) and integers labels (iris.target)
X=iris.data.astype(np.float32)  # Convert to float32 for TensorFlow efficiency /consistency
y=iris.target.astype(np.int32) # Integer class IDs (0..2) suit sparse cross entropy

In [ ]:
# Split into train/test  while preserving original class ratios (stratify)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)#stratify is used to keep class balance
# 20% test gives us an honest final evaluation

In [ ]:
# Scale features using ONLY the training set to avoid data leakage
scaler = StandardScaler()  # Standadization helps optimization (balanced gradients)
X_train = scaler.fit_transform(X_train) # Learn mean/std on train , then scale train
X_test = scaler.transform(X_test) # Scale test with the same train statistics

In [ ]:
# Reshape features to a 1D "signal": (samples , length=4 , channels=1)
X_train = X_train[...,np.newaxis]  # Now shape is (N_train ,4 ,1)
X_test = X_test[...,np.newaxis]  # Now shape is (N_train ,4 ,1)

In [ ]:
# Parts of one layer -  convolutional layer - extract features , cooling layer/operation- useful features selection , activation function

#Define a 1D-CNN with EXACTLY 5 Conv layers
#We use paddind="same"  so length stays 4 throughtout ; kernel_size=2 captures local pairwise interactions.
model = tf.keras.Sequential([         #Stack layers in order
                             tf.keras.layers.Input(shape=(4,1)),# Expext sequence of length 4 with 1 channel
                             tf.keras.layers.Conv1D(16, kernel_size=2, padding="same", activation="relu"), # Conv #1: few filters to start
                             tf.keras.layers.Conv1D(32, kernel_size=2, padding="same", activation="relu"),# Conv #2: widen feature capacity
                             tf.keras.layers.Conv1D(64, kernel_size=2, padding="same", activation="relu"),# Conv #3: deeper , richer features
                             tf.keras.layers.Conv1D(64, kernel_size=2, padding="same", activation="relu"),# Conv #4: maintain depth
                             tf.keras.layers.Conv1D(32, kernel_size=2, padding="same", activation="relu"),# Conv #5: taper down to reduce params
                             tf.keras.layers.GlobalAveragePooling1D(),  #Collpse length  dimension by averaging channels (order-free)
                             tf.keras.layers.Dense(16,activation="relu"), # Small dense head to mix feature non linearly
                             tf.keras.layers.Dense(3,activation="softmax"), # Output probabilities for 3 Iris species
                            ])# kernel - filter - predefined values - different for img , voice etc.

In [ ]:
#Show a concise architecture summary to verify we indeed have 5 Conv layers
model.summary() #Good sanity check for layer counts , shapes  , and parameter sizes

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 4, 16)          │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 4, 32)          │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 4, 64)          │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 4, 32)          │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,227 (71.20 KB)

 Trainable params: 18,227 (71.20 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# === 4) Compile (choose optimizer, loss, and metrics) ========================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),   # Adam is a robust default optimizer
    loss='sparse_categorical_crossentropy',                   # Use integer labels ⇒ “sparse_” CE
    metrics=['accuracy']                                      # Track accuracy during training/eval
)

In [ ]:
# Train with early stopping to avoid overfitting =======================
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, restore_best_weights=True
)  # Stop when validation loss stalls; roll back to the best model

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1   # 👈 shows training & validation accuracy/loss for every epoch
)



ValueError: You must call `compile()` before using the model.

In [ ]:
# === Plot training & validation curves =======================================
plt.figure(figsize=(12,5))

# Plot Loss
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

plt.show()


In [ ]:
# === 6) Evaluate & report ====================================================
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)   # Final unbiased test metrics
print(f"Test accuracy: {test_acc:.3f}")                           # Human-friendly accuracy readout

y_prob = model.predict(X_test, verbose=0)                         # Predicted class probabilities
y_pred = y_prob.argmax(axis=1)                                    # Convert probs → predicted class IDs

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))    # See per-class confusions
print("\nClassification report:\n",                                # Precision/Recall/F1 per class
      classification_report(y_test, y_pred, target_names=iris.target_names))